In [1]:
from datetime import date, datetime, time
import pandas as pd
import numpy as np
import yaml
from sqlalchemy import create_engine, inspect, text

# Extract

In [ ]:
# Cargar configuración desde el archivo config.yml
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_bodega = config['bodega']

# Crear conexión a la bodega de datos
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear SQLAlchemy engine
bodega_conn = create_engine(url_bodega)

# Transformation

In [ ]:
# Crear datos para la dimensión hora

# Definir función para categorizar las horas en períodos del día
def get_periodo_dia(hora):
    if 0 <= hora < 6:
        return 'Madrugada'
    elif 6 <= hora < 12:
        return 'Mañana'
    elif 12 <= hora < 18:
        return 'Tarde'
    else:
        return 'Noche'

In [ ]:
# Generar datos de horas y asignar período del día
horas = []
for hora in range(24):
    horas.append({
        'hora': hora,
        'periodo_dia': get_periodo_dia(hora)
    })

In [5]:
# Crear DataFrame
dim_hora = pd.DataFrame(horas)

In [ ]:
# Agregar columna fecha de carga
dim_hora['saved'] = date.today()

### Verifications

In [7]:
# Verificar los datos
print("\nInformación del DataFrame:")
print(dim_hora.info())

print("\nMuestra de los datos:")
print(dim_hora.head())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   hora         24 non-null     int64 
 1   periodo_dia  24 non-null     object
 2   saved        24 non-null     object
dtypes: int64(1), object(2)
memory usage: 708.0+ bytes
None


# Load

In [9]:
# Cargar en la bodega
dim_hora.to_sql('dim_hora', bodega_conn, if_exists='replace', index_label='key_dim_hora')

24

In [10]:
# Verificar algunos valores
print("\nDistribución de periodos del día:")
print(dim_hora['periodo_dia'].value_counts())


Distribución de periodos del día:
periodo_dia
Madrugada    6
Mañana       6
Tarde        6
Noche        6
Name: count, dtype: int64


In [11]:
print("\nRango de horas:")
print(f"Mínima: {dim_hora['hora'].min()}")
print(f"Máxima: {dim_hora['hora'].max()}")


Rango de horas:
Mínima: 0
Máxima: 23


In [12]:
# Verificación de datos
print("\nEjemplos de diferentes períodos del día:")
for periodo in ['Madrugada', 'Mañana', 'Tarde', 'Noche']:
    print(f"\n{periodo}:")
    print(dim_hora[dim_hora['periodo_dia'] == periodo].head(3))


Ejemplos de diferentes períodos del día:

Madrugada:
   hora periodo_dia       saved
0     0   Madrugada  2024-11-08
1     1   Madrugada  2024-11-08
2     2   Madrugada  2024-11-08

Mañana:
   hora periodo_dia       saved
6     6      Mañana  2024-11-08
7     7      Mañana  2024-11-08
8     8      Mañana  2024-11-08

Tarde:
    hora periodo_dia       saved
12    12       Tarde  2024-11-08
13    13       Tarde  2024-11-08
14    14       Tarde  2024-11-08

Noche:
    hora periodo_dia       saved
18    18       Noche  2024-11-08
19    19       Noche  2024-11-08
20    20       Noche  2024-11-08
